## git repo 연결 (ai)

실행

In [18]:
!git clone https://github.com/na1-4an/SQuiz.git

Cloning into 'SQuiz'...
remote: Enumerating objects: 717, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 717 (delta 33), reused 120 (delta 15), pack-reused 546
Receiving objects: 100% (717/717), 9.44 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (190/190), done.


In [20]:
cd SQuiz

/content/SQuiz


In [21]:
!git checkout origin/ai

Note: switching to 'origin/ai'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ba74363 ➕ Add pdf to text datasets


## pdf to txt

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import os

In [ ]:
def extract_text_from_pdf(pdf_path, txt_path, start_page=None, end_page=None):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""

        if not start_page:
            start_page = 0

        else:
            start_page -= 1

        if not end_page or end_page > len(reader.pages):
            end_page = len(reader.pages)

        for page_num in range(start_page, end_page):
            text += "\n<" + str(page_num) + ">\n"
            text += reader.pages[page_num].extract_text()


    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

In [ ]:
def list_pdf_files_in_directory(directory):
    return [f for f in os.listdir(directory) if f.endswith('.pdf')]

In [ ]:
if __name__ == "__main__":
    #directory = input("pdf 파일 경로를 입력하세요: ")
    # pdf_files = list_pdf_files_in_directory(directory)

    # for idx, pdf_file in enumerate(pdf_files, 1):
    #     print(f"{idx}. {pdf_file}")

    # selection = int(input("이 중에 원하시는 파일의 번호를 골라주세요: "))
    # pdf_path = os.path.join(directory, pdf_files[selection - 1])

    # page_range = input("원하는 페이지 수를 -를 사용하여 입력해주세요. 만약 전체 페이지를 원하시면 ALL을 입력해주세요: ")

    # if page_range != "ALL":
    #     start_page, end_page = map(int, page_range.split('-'))

    # else:
    #     start_page, end_page = None, None

    # txt_directory = input("txt 파일 경로를 입력해주세요: ")
    # txt_filename = input("파일 이름을 확장자를 포함하여 입력해주세요: ")
    # txt_path = os.path.join(txt_directory, txt_filename)

    pdf_path = "/content/제10장 배열(강의).pdf"
    txt_path = "/content/제10장 배열(강의).txt"

    extract_text_from_pdf(pdf_path, txt_path)

    pdf_path = "/content/제11장 포인터(강의).pdf"
    txt_path = "/content/제11장 포인터(강의).txt"
    extract_text_from_pdf(pdf_path, txt_path)


## gpt api setting

1. library 다운로드

2. OpenAI 객체 생성 (API key)


In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

## txt to jsonl (API call)

/SQuiz/test/txt에 있는 txt 파일들 불러오고 공백 제거

In [ ]:
import os

def load_txt(dir_path):

  ## 경로 내의 모든 txt 파일 불러옴
  file_list = os.listdif(dir_path)
  txt_file_list = [file for file in file_list if file.endswith('.txt')]

  txt = [] # 각 pdf의 텍스트 저장할 배열

  ## txt 파일 내의 공백 제거
  for path in txt_file_list:
    with open(path, 'r') as f:
      raw = f.read().replace('\n', ' ') # 개행 문자 -> 공백으로 변경
      txt.append(raw)

  return txt

In [10]:
def create_msg(subject, pdftxt):
  msg = []

  system_json = {"role": "system"}
  system_json["content"] = "You will be provided with a thesis abstract and a suggested title for it. The thesis title should give the reader a good idea of the topic of the thesis but should also be eye-catching. If the title does not meet these criteria, suggest 5 alternatives."

  user_json = {"role": "user"}
  user_json["content"] = pdftxt

  msg.append(system_json)
  msg.append(user_json)

  return msg

In [15]:
def create_req(msg, model="gpt-3.5-turbo", max_token=25, top_p=1, frequency_penalty=0, presence_penalty=0):
  req = {}
  req["model"] = model
  req["message"] = msg
  req["max_tokens"] = max_tokens
  req["top_p"] = top_p
  req["frequency_penalty"] = frequency_penalty
  req["presence_penalty"] = presence_penalty
  return req

In [ ]:
txt = load_txt("/content/SQuiz/test/txt")
model = "gpt-3.5-turbo"
max_tokens=25
top_p=1
frequency_penalty=0
presence_penalty=0

msg = create_msg(0, txt[0])
req = create_req(msg, model, max_tokens, top_p, frequency_penalty, presence_penalty)
#reponse = client.chat.completions.create(req)

response 받기

In [ ]:
msg = create_msg(0, txt[1]) #과목과 pdf2txt 내용을 넣음

response = client.chat.completions.create(
  model='gpt-3.5-turbo',
  message=msg,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

prompt

In [ ]:
#prompt 조정용 코드
completion = client.chat.completions.create(
  model='gpt-3.5-turbo',
  message=[
      ##여기를 수정
      { "role": "system",
        "content": "You will be provided with a thesis abstract and a suggested title for it. The thesis title should give the reader a good idea of the topic of the thesis but should also be eye-catching. If the title does not meet these criteria, suggest 5 alternatives."
      },
      ##여기에는 pdf 내용을 입력 (여기 텍스트 포맷도 수정)
      { "role": "user",
        "content": txt[0]
      }
  ],
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(completion.choices[0].message)

In [ ]:
<0>Introduction1-1Chapter 1IntroductionComputer Networking: A Top DownApproach A note on the use of these Powerpoint slides:We’re making these slides freely available to all (faculty, students, readers). They’re in PowerPoint form so you see the animations; and can add, modify, and delete slides  (including this one) and slide content to suit your needs. They obviously represent a lotof work on our part. In return for use, we only ask the following:§If you use these slides (e.g., in a class) that you mention their source (after all, we’d like people to use our book!)§If you post any slides on a www site, that you note that they are adapted from (or perhaps identical to) our slides, andnote our copyright of this material.Thanksand enjoy!  JFK/KWRAll material copyright 1996-2016J.F Kurose and K.W. Ross, All Rights Reserved7thedition Jim Kurose, Keith RossPearson/Addison WesleyApril 2016<1>IntroductionChapter 1: introductionour goal:§get “feel”and terminology§more depth, detail laterin course§approach:•use Internet as exampleoverview:§what’s the Internet?§what’s a protocol?§network edge; hosts, access net, physical media§network core: packet/circuit switching, Internet structure§performance: loss, delay, throughput§security§protocol layers, service models§history1-2<2>IntroductionChapter 1: roadmap1.1 what isthe Internet?1.2network edge§end systems, access networks, links1.3 network core§packet switching, circuit switching, network structure1.4 delay, loss, throughput in networks1.5protocol layers, service models1.6networks under attack: security1.7history1-3<3>IntroductionWhat’s the Internet: “nuts and bolts”view§billions of connected computing devices: •hosts =end systems•running network apps§communication links•fiber, copper, radio, satellite•transmission rate: bandwidth§packet switches:forward packets (chunks of data)•routersand switcheswiredlinkswirelesslinksroutersmartphonePCserverwirelesslaptop1-4mobile networkglobal ISPregional ISPhome networkinstitutionalnetwork<4>Introduction“Fun”Internet-connected devicesIP picture framehttp://www.ceiva.com/Web-enabled toaster +weather forecasterInternet phonesInternet refrigeratorSlingbox: watch,control cable TV remotely1-5Tweet-a-watt: monitor energy usesensorized,bedmattress<5>Introduction§Internet: “network of networks”•Interconnected ISPs§protocolscontrol sending, receiving of messages•e.g., TCP, IP, HTTP, Skype,  802.11§Internet  standards•RFC: Request for comments•IETF: Internet Engineering Task ForceWhat’s the Internet: “nuts and bolts”view1-6mobile networkglobal ISPregional ISPhome networkinstitutionalnetwork<6>What’s the Internet: a service view§infrastructure that provides services to applications:•Web, VoIP, email, games, e-commerce, social nets, …§provides programming interface to apps•hooks that allow sending and receiving  app programs to “connect”to Internet•provides service options, analogous to postal serviceIntroduction1-7mobile networkglobal ISPregional ISPhome networkinstitutionalnetwork<7>IntroductionWhat’s a protocol?human protocols:§“what’s the time?”§“I have a question”§introductions… specific messages sent… specific actions taken when messages received, or other eventsnetwork protocols:§machines rather than humans§all communication activity in Internet governed by protocolsprotocolsdefine format, orderof messages sent and receivedamong network entities, and actions takenon message transmission, receipt1-8<8>Introductiona human protocol and a computer network protocol:Q:other human protocols? HiHiGot thetime?2:00TCP connectionresponseGethttp://www.awl.com/kurose-ross<file>timeTCP connectionrequestWhat’s a protocol?1-9<9>IntroductionChapter 1: roadmap1.1 what isthe Internet?1.2 network edge§end systems, access networks, links1.3 network core§packet switching, circuit switching, network structure1.4 delay, loss, throughput in networks1.5protocol layers, service models1.6networks under attack: security1.7history1-10<10>IntroductionA closer look at network structure:§network edge:•hosts: clients and servers•servers often in data centers§access networks, physical media:wired, wireless communication links§network core: •interconnected routers•network of networks1-11mobile networkglobal ISPregional ISPhome networkinstitutionalnetwork<11>IntroductionAccess networks and physical mediaQ: How to connect end systems to edge router?§residential access nets§institutional access networks (school, company)§mobile access networkskeep in mind: §bandwidth (bits per second) of access network?§shared or dedicated?1-12<12>ISPIntroductionAccess network: digital subscriber line (DSL)central officetelephonenetworkDSLAMvoice, data transmittedat different frequencies overdedicated line to central office§use existingtelephone line to central office DSLAM•data over DSL phone line goes to Internet•voice over DSL phone line goes to telephone net§< 2.5 Mbps upstream transmission rate (typically < 1 Mbps)§< 24 Mbps downstream transmission rate (typically < 10 Mbps)DSLmodemsplitterDSL access multiplexer1-13<13>IntroductionAccess network: cable networkcablemodemsplitter…cable headendChannelsVIDEOVIDEOVIDEOVIDEOVIDEOVIDEODATADATACONTROL123456789frequency division multiplexing:different channels transmittedin different frequency bands1-14<14>ISPIntroductiondata, TV transmitted at different frequencies over shared cable distribution networkcablemodemsplitter…cable headendCMTScable modemtermination system§HFC: hybrid fiber coax•asymmetric: up to 30Mbps downstream transmission rate, 2 Mbps upstream transmission rate§network of cable, fiber attaches homes to ISP router•homes share access networkto cable headend •unlike DSL, which has dedicated access to central officeAccess network: cable network1-15<15>IntroductionAccess network: home networkto/from headend or central officecable or DSL modemrouter, firewall, NATwired Ethernet (1 Gbps)wireless access point (54 Mbps)wirelessdevicesoften combined in single box1-16<16>IntroductionEnterprise access networks (Ethernet)§typically used in companies, universities, etc.§10 Mbps, 100Mbps, 1Gbps, 10Gbps transmission rates§today, end systems typically connect into Ethernet switchEthernet switchinstitutional mail,web serversinstitutional routerinstitutional link to ISP (Internet)1-17<17>IntroductionWireless access networks§shared wirelessaccess network connects end system to router•via base station aka “access point”wireless LANs:§within building (100 ft.)§802.11b/g/n (WiFi): 11, 54, 450 Mbps transmission ratewide-area wireless access§provided by telco (cellular) operator, 10’s km§between 1 and 10 Mbps §3G, 4G: LTEto Internetto Internet1-18<18>Host: sends packetsof datahost sending function:§takes application message§breaks into smaller chunks, known as packets, of length Lbits§transmits packet into access network at transmission rate R•link transmission rate, aka link capacity, aka link bandwidthR: link transmission ratehost12two packets, Lbits eachpackettransmissiondelaytime needed totransmit L-bitpacket into linkL(bits)R(bits/sec)==1-19Introduction<19>IntroductionPhysical media§bit:propagates betweentransmitter/receiver pairs§physical link:what lies between transmitter & receiver§guided media: •signals propagate in solid media: copper, fiber, coax§unguided media:•signals propagate freely, e.g., radiotwisted pair (TP)§two insulated copper wires•Category 5: 100 Mbps, 1 Gbps Ethernet•Category 6: 10Gbps1-20<20>IntroductionPhysical media: coax, fibercoaxial cable:§two concentric copper conductors§bidirectional§broadband:•multiple channels on cable•HFCfiber optic cable:§glass fiber carrying light pulses, each pulse a bit§high-speed operation:•high-speed point-to-point transmission (e.g., 10’s-100’s Gbps transmission rate)§low error rate: •repeaters spaced far apart •immune to electromagnetic noise1-21<21>IntroductionPhysical media: radio§signal carried in electromagnetic spectrum§no physical “wire”§bidirectional§propagation environment effects:•reflection •obstruction by objects•interferenceradio link types:§terrestrial  microwave•e.g. up to 45 Mbps channels§LAN(e.g., WiFi)•54 Mbps§wide-area(e.g., cellular)•4G cellular: ~ 10 Mbps§satellite•Kbps to 45Mbps channel (or multiple smaller channels)•270 msec end-end delay•geosynchronous versus low altitude1-22<22>IntroductionChapter 1: roadmap1.1 what isthe Internet?1.2network edge§end systems, access networks, links1.3 network core§packet switching, circuit switching, network structure1.4 delay, loss, throughput in networks1.5protocol layers, service models1.6networks under attack: security1.7history1-23<23>Introduction§mesh of interconnected routers§packet-switching: hosts break application-layer messages into packets•forward packetsfrom one router to the next, across links on path from source to destination•each packet transmitted at full link capacityThe network core1-24<24>IntroductionPacket-switching: store-and-forward§takes L/Rseconds to transmit (push out) L-bit packet into link at Rbps§store and forward:entire packet must  arrive at router before it can be transmitted on next linkone-hop numerical example:§L= 7.5 Mbits§R= 1.5 Mbps§one-hop transmission delay = 5 secmore on delay shortly …1-25sourceRbpsdestination123Lbitsper packetRbps§end-end delay = 2L/R(assuming zero propagation delay)<25>IntroductionPacket Switching: queueing delay, lossABCR= 100 Mb/sR= 1.5 Mb/sDEqueue of packetswaiting for output link1-26queuing and loss: §if arrival rate (in bits) to link exceeds transmission rate of link for a period of time:•packets will queue, wait to be transmitted on link •packets can be dropped (lost) if memory (buffer) fills up<26>Two key network-core functionsforwarding:move packets from router’s input to appropriate router outputIntroduction1-27routing:determines source-destination route taken by packets§routing algorithmsrouting algorithmlocal forwarding tableheader valueoutput link010001010111100132211230111destination address in arrivingpacket’s header<27>IntroductionAlternative core: circuit switchingend-end resources allocated to, reserved for “call”between source & dest:§in diagram, each link has four circuits. •call gets 2ndcircuit in top link and 1stcircuit in right link.§dedicated resources: no sharing•circuit-like (guaranteed) performance§circuit segment idle if not used by call (no sharing)§commonly used in traditional telephone networks1-28<28>IntroductionCircuit switching: FDM versusTDMFDMfrequencytimeTDMfrequencytime4 usersExample:1-29<29>IntroductionPacket switching versus circuit switchingexample:§1 Mb/s link§each user: •100 kb/s when “active”•active 10% of time§circuit-switching:•10 users§packet switching:•with 35 users, probability > 10 active at same time is less than .0004 *packet switching allows more users to use network!Nusers1 Mbps linkQ:how did we get value 0.0004?Q:what happens if > 35 users ?…..1-30* Check out the online interactive exercises for more examples: http://gaia.cs.umass.edu/kurose_ross/interactive/<30>Introduction§great for bursty data•resource sharing•simpler, no call setup§excessive congestion possible:packet delay and loss•protocols needed for reliable data transfer, congestion control§Q:How to provide circuit-like behavior?•bandwidth guarantees needed for audio/video apps•still an unsolved problem (chapter 7)is packet switching a “slam dunk winner?”Q:human analogies of reserved resources (circuit switching) versus on-demand allocation (packet-switching)?Packet switching versus circuit switching1-31<31>Internet structure: network of networks§End systems connect to Internet via access ISPs (Internet Service Providers)•residential, company and university ISPs§Access ISPs in turn must be interconnected. •so that any two hosts can send packets to each other§Resulting network of networks is very complex•evolution was driven by economicsand national policies§Let’s take a stepwise approach to describe current Internet structureIntroduction1-32<32>Internet structure: network of networksQuestion: given millionsof access ISPs, how to connect them together?accessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnet………………Introduction1-33<33>Internet structure: network of networksOption: connect each access ISP to every other access ISP? accessnetaccessnet……………………connecting each access ISP to each other directly doesn’t scale: O(N2) connections.Introduction1-34accessnetaccessnetaccessnetaccessnetaccessnetaccessnet…accessnetaccessnetaccessnetaccessnet…accessnetaccessnetaccessnetaccessnet…<34>Internet structure: network of networksaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnet………………Option: connect each access ISP to one global transit ISP? Customerand provider ISPs have economic agreement.Introduction1-35globalISP<35>ISP CISP BISP AInternet structure: network of networksaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnet………………But if one global ISP is viable business, there will be competitors ….Introduction1-36accessnet<36>ISP CISP BISP AInternet structure: network of networksaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnet………………Introduction1-37accessnetBut if one global ISP is viable business, there will be competitors ….  which must be interconnectedIXPpeering linkInternet exchange point IXP<37>ISP CISP BISP AInternet structure: network of networksaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnet………………Introduction1-38accessnetIXPIXPaccessnetaccessnetaccessnetregional net… and regional networks may arise to connect access nets to ISPs <38>ISP CISP BISP AInternet structure: network of networksaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnetaccessnet………………Introduction1-39accessnetIXPIXPaccessnetaccessnetaccessnetregional netContent provider network… and content provider networks  (e.g., Google, Microsoft,   Akamai) may run their own network, to bring services, content close to end users<39>IntroductionInternet structure: network of networks§at center: small # of well-connected large networks•“tier-1”commercial ISPs(e.g., Level 3, Sprint, AT&T, NTT), national & international coverage•content provider network (e.g., Google): private network that connects it data centers to Internet, often bypassing tier-1, regional ISPs1-40IXPIXPIXPTier 1 ISPTier 1 ISPGoogleRegional ISPRegional ISPaccessISPaccessISPaccessISPaccessISPaccessISPaccessISPaccessISPaccessISP<40>IntroductionTier-1 ISP: e.g., Sprint1-41…to/from customerspeeringto/from backbone…………POP: point-of-presence<41>IntroductionChapter 1: roadmap1.1 what isthe Internet?1.2network edge§end systems, access networks, links1.3 network core§packet switching, circuit switching, network structure1.4 delay, loss, throughput in networks1.5protocol layers, service models1.6networks under attack: security1.7history1-42<42>ABIntroductionHow do loss and delay occur?packets queuein router buffers§packet arrival rate to link (temporarily) exceeds output link capacity§packets queue, wait for turnpacket being transmitted (delay)packets queueing(delay)free (available) buffers: arriving packets dropped (loss) if no free buffers1-43<43>IntroductionFour sources of packet delaydproc: nodal processing§check bit errors§determine output link§typically < msecdqueue: queueing delay§time waiting at output link for transmission §depends on congestion level of router1-44propagationnodalprocessingqueueingdnodal= dproc+ dqueue+ dtrans+  dpropABtransmission<44>Introductiondtrans: transmission delay:§L: packet length (bits) §R: link bandwidth (bps)§dtrans= L/Rdprop: propagation delay:§d: length of physical link§s: propagation speed (~2x108m/sec)§dprop= d/sFour sources of packet delay1-45* Check out the Java applet for an interactive animation on trans vs. prop delaydtrans and dpropvery different* Check out the online interactive exercises for more examples: http://gaia.cs.umass.edu/kurose_ross/interactive/propagationnodalprocessingqueueingdnodal= dproc+ dqueue+ dtrans+  dpropABtransmission<45>IntroductionCaravan analogy§cars “propagate”at 100 km/hr§toll booth takes 12 sec to service car (bit transmission time)§car ~ bit; caravan ~ packet§Q:How long until caravan is lined up before 2nd toll booth?§time to “push”entire caravan through toll booth onto highway = 12*10 = 120 sec§time for last car to propagate from 1st to 2nd toll both: 100km/(100km/hr)= 1 hr§A:62 minutestoll boothtoll boothten-car caravan100 km100 km1-46<46>IntroductionCaravan analogy (more)§suppose cars now “propagate”at 1000 km/hr§and suppose toll booth now takes one min to service a car§Q:Will cars arrive to 2nd booth before all cars serviced at first booth?•A: Yes!after 7 min, first car arrives at second booth; three cars still at first boothtoll boothtoll boothten-car caravan100 km100 km1-47<47>Introduction§R:link bandwidth (bps)§L:packet length (bits)§a: average packet arrival ratetraffic intensity = La/R§La/R~ 0: avg. queueing delay small§La/R -> 1: avg. queueing delay large§La/R > 1: more “work”arriving than can be serviced, average delay infinite!average  queueing delayLa/R ~ 0La/R -> 11-48* Check online interactive animation on queuing and lossQueueing delay (revisited)<48>Introduction“Real”Internet delays and routes§what do “real”Internet delay & loss look like? §tracerouteprogram: provides delay measurement from source to router along end-end Internet path towards destination.  For all i:•sends three packets that will reach router ion path towards destination•router iwill return packets to sender•sender times interval between transmission and reply.3 probes3 probes3 probes1-49<49>Introduction“Real”Internet delays, routes1  cs-gw (128.119.240.254)  1 ms  1 ms  2 ms2  border1-rt-fa5-1-0.gw.umass.edu (128.119.3.145)  1 ms  1 ms  2 ms3  cht-vbns.gw.umass.edu (128.119.3.130)  6 ms 5 ms 5 ms4  jn1-at1-0-0-19.wor.vbns.net (204.147.132.129)  16 ms 11 ms 13 ms 5  jn1-so7-0-0-0.wae.vbns.net (204.147.136.136)  21 ms 18 ms 18 ms 6  abilene-vbns.abilene.ucaid.edu (198.32.11.9)  22 ms  18 ms  22 ms7  nycm-wash.abilene.ucaid.edu (198.32.8.46)  22 ms  22 ms  22 ms8  62.40.103.253 (62.40.103.253)  104 ms 109 ms 106 ms9  de2-1.de1.de.geant.net (62.40.96.129)  109 ms 102 ms 104 ms10  de.fr1.fr.geant.net (62.40.96.50)  113 ms 121 ms 114 ms11  renater-gw.fr1.fr.geant.net (62.40.103.54)  112 ms  114 ms  112 ms12  nio-n2.cssi.renater.fr (193.51.206.13)  111 ms  114 ms  116 ms13  nice.cssi.renater.fr (195.220.98.102)  123 ms  125 ms  124 ms14  r3t2-nice.cssi.renater.fr (195.220.98.110)  126 ms  126 ms  124 ms15  eurecom-valbonne.r3t2.ft.net (193.48.50.54)  135 ms  128 ms  133 ms16  194.214.211.25 (194.214.211.25)  126 ms  128 ms  126 ms17  * * *18  * * *19  fantasia.eurecom.fr (193.55.113.142)  132 ms  128 ms  136mstraceroute:gaia.cs.umass.edu to www.eurecom.fr3 delay measurements from gaia.cs.umass.edu to cs-gw.cs.umass.edu * means no response (probe lost, router not replying)trans-oceaniclink1-50* Do some traceroutes from exotic countries at www.traceroute.org<50>IntroductionPacket loss§queue (aka buffer) preceding link in buffer has finite capacity§packet arriving to full queue dropped (aka lost)§lost packet may be retransmitted by previous node, by source end system, or not at allABpacket being transmittedpacket arriving tofull buffer is lostbuffer (waiting area)1-51* Check out the Java applet for an interactive animation on queuing and loss<51>IntroductionThroughput§throughput:rate (bits/time unit) at which bits transferred between sender/receiver•instantaneous:rate at given point in time•average:rate over longer period of timeserver, withfile of F bits to send to clientlink capacityRsbits/seclink capacityRcbits/secserver sends bits (fluid) into pipepipe that can carryfluid at rateRsbits/sec)pipe that can carryfluid at rateRcbits/sec)1-52<52>IntroductionThroughput (more)§Rs< RcWhat is average end-end throughput?Rsbits/secRcbits/sec§Rs> RcWhat is average end-end throughput?link on end-end path that constrains  end-end throughputbottleneck linkRsbits/secRcbits/sec1-